# Analysis of the Chandra Data of MACS J0717.5+3745

### 1. Reprocess the Chandra ObsIDs

Define the ObsIDs used in the analysis.

In [1]:
obsid1="16235"
obsid2="16305"
obsid3="4200"
obsid4="1655"

Reprocess each dataset to create new level=2 event files with the newest calibration applied. We used CIAO 4.8.2 and CALDB 4.7.0. In <i>chandra_repro</i>, the parameter `check_vf_pha` is set by default to `no`. This parameter should be set to `yes` if an observation was taken in VFAINT mode, so one needs to check the mode of each ObsID.

In [2]:
punlearn ardlib

In [ ]:
for obsid in $obsid1 $obsid2 $obsid3 $obsid4
do
    mode_obsid=`dmkeypar ${obsid}/primary/*${obsid}*evt2.fits.gz DATAMODE echo+`
    rm -rf ${obsid}/repro
    mkdir ${obsid}/repro

    if [ $mode_obsid == 'VFAINT' ]
    then
        chandra_repro indir=$obsid outdir=$obsid/repro check_vf_pha=yes clobber+
    else
        chandra_repro indir=$obsid outdir=$obsid/repro clobber+
    fi
done

### 2. Flare-Filter the Data

The flare filtering was done in the energy band 0.5-7 keV. It consists of two steps:
<ol>
<li>Bright sources are detected and removed from the event files. The point source detection done at this point can be less thorough than it will be done later on.</li>
<li>Flares are detected in light curves free of bright sources.</li>
</ol>

In [7]:
punlearn ardlib
punlearn mkpsfmap
punlearn wavdetect
punlearn fluximage

for obsid in $obsid1 $obsid2 $obsid3 $obsid4
do
    acis_set_ardlib ${obsid}/repro/*_repro_bpix1.fits
    len=${#obsid}

    if [ ${len} -eq 4 ]
    then
        fluximage infile=${obsid}/repro/acisf0${obsid}_repro_evt2.fits \
            outroot=${obsid}/repro/${obsid} bands=0.5:7:2.3 binsize=4 clobber+
    else
        fluximage infile=${obsid}/repro/acisf${obsid}_repro_evt2.fits \
            outroot=${obsid}/repro/${obsid} bands=0.5:7:2.3 binsize=4 clobber+
    fi

    mkpsfmap ${obsid}/repro/${obsid}_0.5-7_thresh.img ${obsid}/repro/${obsid}_0.5-7keV.psf \
        energy=2.3 ecf=0.9 clobber+

    wavdetect infile=${obsid}/repro/${obsid}_0.5-7_thresh.img psffile=${obsid}/repro/${obsid}_0.5-7keV.psf \
        expfile=${obsid}/repro/${obsid}_0.5-7_thresh.expmap outfile=${obsid}/repro/src_0.5-7keV.fits \
        scellfile=${obsid}/repro/scell_0.5-7keV.fits imagefile=${obsid}/repro/src_0.5-7keV.img \
        defnbkgfile=${obsid}/repro/nbkg_0.5-7keV.fits regfile=${obsid}/repro/${obsid}_src_0.5-7keV.reg \
        scales="1.0 2.0 4.0 8.0 16.0 32.0" sigthresh=5e-6 ellsigma=5.0 clobber+
done

Updated ardlib parameter file: /Users/gogrean/cxcds_param4/ardlib.par
  AXAF_ACIS0_BADPIX_FILE -> /Users/gogrean/data/chandra/macsj0717/16235/repro/acisf16235_repro_bpix1.fits[BADPIX0]
  AXAF_ACIS1_BADPIX_FILE -> /Users/gogrean/data/chandra/macsj0717/16235/repro/acisf16235_repro_bpix1.fits[BADPIX1]
  AXAF_ACIS2_BADPIX_FILE -> /Users/gogrean/data/chandra/macsj0717/16235/repro/acisf16235_repro_bpix1.fits[BADPIX2]
  AXAF_ACIS3_BADPIX_FILE -> /Users/gogrean/data/chandra/macsj0717/16235/repro/acisf16235_repro_bpix1.fits[BADPIX3]
  AXAF_ACIS4_BADPIX_FILE -> CALDB
  AXAF_ACIS5_BADPIX_FILE -> CALDB
  AXAF_ACIS6_BADPIX_FILE -> CALDB
  AXAF_ACIS7_BADPIX_FILE -> CALDB
  AXAF_ACIS8_BADPIX_FILE -> CALDB
  AXAF_ACIS9_BADPIX_FILE -> CALDB
Running fluximage
Version: 05 January 2016

Using energy range 0.5 to 7.0 keV and exposure-map energy of 2.3 keV.
Aspect solution 16235/repro/pcadf503361159N002_asol1.fits found.
Bad-pixel file 16235/repro/acisf16235_repro_bpix1.fits found.
Mask file 16235/repro/aci

The region files created by <i>wavdetect</i> have an additional region (circle) added to them to exclude emission from the cluster. The final region files are saved in physical coordinates as:
* 1655_src_0.5-7keV_phys.reg
* 4200_src_0.5-7keV_phys.reg
* 16235_src_0.5-7keV_phys.reg
* 16305_src_0.5-7keV_phys.reg

and are available in the corresponding `<ObsID>/repro` directories available in the Github repository.

Next, the bright sources are masked in the event files, and flares are detected and removed from all ObsIDs.

In [9]:
punlearn dmcopy
punlearn dmextract
punlearn deflare

for obsid in $obsid1 $obsid2 $obsid3 $obsid4
do
    len=${#obsid}

    if [ $len -eq 4 ]
    then
        dmcopy "${obsid}/repro/acisf0${obsid}_repro_evt2.fits[exclude sky=region(${obsid}/repro/${obsid}_src_0.5-7keV_phys.reg)]" \
            ${obsid}/repro/${obsid}_nosources.evt option=all clobber+
    else
        dmcopy "${obsid}/repro/acisf${obsid}_repro_evt2.fits[exclude sky=region(${obsid}/repro/${obsid}_src_0.5-7keV_phys.reg)]" \
            ${obsid}/repro/${obsid}_nosources.evt option=all clobber+
    fi

    dmcopy "${obsid}/repro/${obsid}_nosources.evt[energy=500:7000]" \
        ${obsid}/repro/${obsid}_0.5-7keV_nosources.evt option=all clobber+

    dmextract infile="${obsid}/repro/${obsid}_0.5-7keV_nosources.evt[bin time=::259.28]" \
        outfile=${obsid}/repro/${obsid}_0.5-7keV.lc opt=ltc1 clobber+

    deflare ${obsid}/repro/${obsid}_0.5-7keV.lc ${obsid}/repro/${obsid}_0.5-7keV.gti method=clean

    if [ $len -eq 4 ]
    then
        dmcopy "${obsid}/repro/acisf0${obsid}_repro_evt2.fits[@${obsid}/repro/${obsid}_0.5-7keV.gti]" \
            ${obsid}/repro/acisf${obsid}_clean.evt opt=all clobber+
    else
        dmcopy "${obsid}/repro/acisf${obsid}_repro_evt2.fits[@${obsid}/repro/${obsid}_0.5-7keV.gti]" \
            ${obsid}/repro/acisf${obsid}_clean.evt opt=all clobber+
    fi
done

Parameters used to clean the lightcurve are:
  script version = November 2015
  mean           = None
  clip           = 3
  scale          = 1.2
  minfrac        = 0.1
  outfile        = 16235/repro/16235_0.5-7keV.gti
  plot           = False

Total number of bins in lightcurve   = 283
Max length of one bin                = 255.892 s
Num. bins with a smaller exp. time   = 3
Num. bins with exp. time = 0         = 8
Number of bins with a rate of 0 ct/s = 8

Calculated an initial mean (sigma-clipped) rate of 0.299467 ct/s
Lightcurve limits use a scale factor of 1.2 about this mean
Filtering lightcurve between rates of 0.249556 and 0.35936 ct/s
Number of good time bins = 241
Rate filter:  0.24955569181320911 <= count_rate < 0.35936019621102105 
Mean level of filtered lightcurve = 0.30111668889722126 ct/s

GTI limits calculated using a count-rate filter:
  (count_rate>0.24955569181320911 && count_rate<0.35936019621102105)

The corresponding times are:
  ((time >= 503361935.95081002) && (ti

### 3. Reproject ObsIDs

The flare-filtered event files were reprojected to a common reference frame. A merged event file that combines the individual event files is also created.

In [11]:
punlearn reproject_obs

pcad_obsid1=`find ${obsid1}/repro -name "pcadf*_asol1.fits"`
pcad_obsid2=`find ${obsid2}/repro -name "pcadf*_asol1.fits"`
pcad_obsid3=`find ${obsid3}/repro -name "pcadf*_asol1.fits"`
pcad_obsid4=`find ${obsid4}/repro -name "pcadf*_asol1.fits"`

mkdir merged4
reproject_obs "${obsid1}/repro/acisf${obsid1}_clean.evt,\
               ${obsid2}/repro/acisf${obsid2}_clean.evt,\
               ${obsid3}/repro/acisf${obsid3}_clean.evt,\
               ${obsid4}/repro/acisf${obsid4}_clean.evt" ./merged4/ \
    asolfiles="${pcad_obsid1},${pcad_obsid2},${pcad_obsid3},${pcad_obsid4}" \
    merge=yes clobber+

mkdir: merged4: File exists
Running reproject_obs
Version: 12 November 2015

Verifying 4 observations.
Calculating new tangent point.
New tangent point: RA=7h 17m 29.28s Dec=37d 46' 11.064"

Observations to be reprojected:

  Obsid  Obs Date   Exp    DETNAM     SIM_Z    FP   Sepn   PA  
                   (ks)                (mm)    (K)   (')  (deg)
---------------------------------------------------------------
1 1655  2001-01-29  15.8 ACIS-01236  -225.843 153.3   3.5  -149
2 4200  2003-01-08  51.6 ACIS-01236  -241.794 154.1   3.0   +97
3 16305 2013-12-11  79.8 ACIS-0123   -225.843 156.0   2.6   -20
4 16235 2013-12-13  61.6 ACIS-0123   -225.843 155.5   1.0   -17

WARNING - DATAMODE values differ:
  Obsids 1655 16235 have DATAMODE=FAINT and the rest have VFAINT

Running tasks in parallel with 4 processors.
Reprojecting 4 event files to a common tangent point.
Merging reprojected events files to ./merged4/merged_evt.fits

The following files were created:

The reprojected event files:
 

### 4. Make Stowed Background Event Files

Stowed background event files appropriate for each ObsID were created from the individual CCD stowed background event files distributed with the Chandra CalDB. The stowed background event files were then reprojected to match the reprojected ObsIDs, and additionally filtered if they were associated with an observation taken in VFAINT mode.

In [14]:
/bin/cp -f /Users/gogrean/astrosoft/ciao/ciao-4.8/CALDB/data/chandra/acis/bkgrnd/acis?D2009*bgstow_cti*.fits \
           ${obsid1}/repro/
/bin/cp -f /Users/gogrean/astrosoft/ciao/ciao-4.8/CALDB/data/chandra/acis/bkgrnd/acis?D2009*bgstow_cti*.fits \
           ${obsid2}/repro/
/bin/cp -f /Users/gogrean/astrosoft/ciao/ciao-4.8/CALDB/data/chandra/acis/bkgrnd/acis?D2000*bgstow_cti*.fits \
           ${obsid3}/repro/
/bin/cp -f /Users/gogrean/astrosoft/ciao/ciao-4.8/CALDB/data/chandra/acis/bkgrnd/acis?D2000*bgstow_cti*.fits \
           ${obsid4}/repro/

In [22]:
punlearn dmmerge
punlearn dmmakepar
punlearn reproject_events

for obsid in ${obsid1} ${obsid2}
do
    dmmerge "${obsid}/repro/acis0D2009-09-21bgstow_ctiN0001.fits,\
             ${obsid}/repro/acis1D2009-09-21bgstow_ctiN0001.fits,\
             ${obsid}/repro/acis2D2009-09-21bgstow_ctiN0001.fits,\
             ${obsid}/repro/acis3D2009-09-21bgstow_ctiN0001.fits" \
             ${obsid}/repro/acisD2009_bgstow.evt clobber+
    dmmakepar merged4/${obsid}_reproj_evt.fits merged4/event_header.par clobber+
    grep _pnt merged4/event_header.par > merged4/event_pnt.par
    chmod +w ${obsid}/repro/acisD2009_bgstow.evt
    dmreadpar merged4/event_pnt.par \
        "${obsid}/repro/acisD2009_bgstow.evt[events]" clobber=yes
    reproject_events infile=${obsid}/repro/acisD2009_bgstow.evt \
        outfile=merged4/${obsid}_bgstow.evt aspect="@${obsid}.aspect" \
        match=merged4/${obsid}_reproj_evt.fits random=0 clobber+
    rm -f merged4/event_header.par
done

for obsid in ${obsid3} ${obsid4}
do
    dmmerge "${obsid}/repro/acis0D2000-12-01bgstow_ctiN0004.fits,\
             ${obsid}/repro/acis1D2000-12-01bgstow_ctiN0004.fits,\
             ${obsid}/repro/acis2D2000-12-01bgstow_ctiN0002.fits,\
             ${obsid}/repro/acis3D2000-12-01bgstow_ctiN0004.fits,\
             ${obsid}/repro/acis6D2000-12-01bgstow_ctiN0004.fits" \
             ${obsid}/repro/acisD2000_bgstow.evt clobber+
    dmmakepar merged4/${obsid}_reproj_evt.fits merged4/event_header.par clobber+
    grep _pnt merged4/event_header.par > merged4/event_pnt.par
    chmod +w ${obsid}/repro/acisD2000_bgstow.evt
    dmreadpar merged4/event_pnt.par \
        "${obsid}/repro/acisD2000_bgstow.evt[events]" clobber=yes
    reproject_events infile=${obsid}/repro/acisD2000_bgstow.evt \
        outfile=merged4/${obsid}_bgstow.evt aspect="@${obsid}.aspect" \
        match=merged4/${obsid}_reproj_evt.fits random=0 clobber+
    rm -f merged4/event_header.par
done

punlearn dmcopy
for obsid in $obsid1 $obsid2 $obsid3 $obsid4
do
    mode_obsid=`dmkeypar ${obsid}/primary/*${obsid}*evt2.fits.gz DATAMODE echo+`
    if [ ${mode_obsid} == 'VFAINT' ]
    then
        echo "${obsid} taken in ${mode_obsid} mode."
        dmcopy "merged4/${obsid}_bgstow.evt[status=0]" \
        merged4/${obsid}_bgstow_clean.evt clobber+
    else
        cp merged4/${obsid}_bgstow.evt merged4/${obsid}_bgstow_clean.evt
        echo "${obsid} taken in ${mode_obsid} mode."
    fi
done

16235 taken in FAINT mode.
16305 taken in VFAINT mode.
4200 taken in VFAINT mode.
1655 taken in FAINT mode.


### 5. Detect Point Sources

(a) We created images and exposure maps in 0.5-2 and 2-7 keV. These are the energy bands in which point sources were later detected.

In [24]:
punlearn flux_obs

flux_obs "merged4/${obsid1}_reproj_evt.fits,\
          merged4/${obsid2}_reproj_evt.fits,\
          merged4/${obsid3}_reproj_evt.fits,\
          merged4/${obsid4}_reproj_evt.fits" \
    merged4/ bands=0.5:2.0:1.5 binsize=2 \
    maskfiles="merged4/${obsid1}.mask,merged4/${obsid2}.mask,\
               merged4/${obsid3}.mask,merged4/${obsid4}.mask" \
    badpixfiles="merged4/${obsid1}.bpix,merged4/${obsid2}.bpix,\
                 merged4/${obsid3}.bpix,merged4/${obsid4}.bpix" \
    expmapthresh="3%" clobber+

flux_obs "merged4/${obsid1}_reproj_evt.fits,\
          merged4/${obsid2}_reproj_evt.fits,\
          merged4/${obsid3}_reproj_evt.fits,\
          merged4/${obsid4}_reproj_evt.fits" \
    merged4/ bands=2.0:7.0:3.8 binsize=2 \
    maskfiles="merged4/${obsid1}.mask,merged4/${obsid2}.mask,\
               merged4/${obsid3}.mask,merged4/${obsid4}.mask" \
    badpixfiles="merged4/${obsid1}.bpix,merged4/${obsid2}.bpix,\
                 merged4/${obsid3}.bpix,merged4/${obsid4}.bpix" \
    expmapthresh="3%" clobber+

Running flux_obs
Version: 05 January 2016

Verifying 4 observations.
Using energy range 0.5 to 2.0 keV and exposure-map energy of 1.5 keV.
Calculating the output grid

The merged images will have 1868 by 2308 pixels, a pixel size of 0.984 arcsec,
    and cover x=2378.5:6114.5:2, y=1266.5:5882.5:2.

Creating the fluxed images for 4 observations in parallel.
Creating aspect histograms for obsid 1655
Creating aspect histograms for obsid 16235
Creating aspect histograms for obsid 4200
Creating aspect histograms for obsid 16305
Creating 4 instrument maps for obsid 16235
Creating 5 instrument maps for obsid 1655
Creating 4 instrument maps for obsid 16305
Creating 5 instrument maps for obsid 4200
Creating 4 exposure maps for obsid 16235
Creating 5 exposure maps for obsid 1655
Creating 4 exposure maps for obsid 16305
Creating 5 exposure maps for obsid 4200
Combining 4 exposure maps for obsid 16235
Combining 5 exposure maps for obsid 1655
Combining 4 exposure maps for obsid 16305
Thresholding d

(b) We created PSF maps for each of the ObsIDs, and combined them into an exposure map-weighed merged PSF map.

In [25]:
punlearn mkpsfmap
for obsid in ${obsid1} ${obsid2} ${obsid3} ${obsid4}
do
    mkpsfmap merged4/${obsid}_0.5-2.0_thresh.img \
        merged4/${obsid}_0.5-2.0.psf energy=1.5 ecf=0.9 clobber+
    mkpsfmap merged4/${obsid}_2.0-7.0_thresh.img \
        merged4/${obsid}_2.0-7.0.psf energy=3.8 ecf=0.9 clobber+
done

dmimgcalc "merged4/${obsid1}_0.5-2.0_thresh.expmap,\
           merged4/${obsid2}_0.5-2.0_thresh.expmap,\
           merged4/${obsid3}_0.5-2.0_thresh.expmap,\
           merged4/${obsid4}_0.5-2.0_thresh.expmap,\
           merged4/${obsid1}_0.5-2.0.psf,merged4/${obsid2}_0.5-2.0.psf,\
           merged4/${obsid3}_0.5-2.0.psf,merged4/${obsid4}_0.5-2.0.psf" \
    none merged4/expweighted_0.5-2.0.psf \
    op="imgout=((img5*img1)+(img6*img2)+(img7*img3)+(img8*img4))/(img1+img2+img3+img4)" clobber+
dmhedit merged4/expweighted_0.5-2.0.psf file= op=add key=BUNIT value="arcsec"

dmimgcalc "merged4/${obsid1}_2.0-7.0_thresh.expmap,\
           merged4/${obsid2}_2.0-7.0_thresh.expmap,\
           merged4/${obsid3}_2.0-7.0_thresh.expmap,\
           merged4/${obsid4}_2.0-7.0_thresh.expmap,\
           merged4/${obsid1}_2.0-7.0.psf,merged4/${obsid2}_2.0-7.0.psf,\
           merged4/${obsid3}_2.0-7.0.psf,merged4/${obsid4}_2.0-7.0.psf" \
    none merged4/expweighted_2.0-7.0.psf \
    op="imgout=((img5*img1)+(img6*img2)+(img7*img3)+(img8*img4))/(img1+img2+img3+img4)" clobber+
dmhedit merged4/expweighted_2.0-7.0.psf file= op=add key=BUNIT value="arcsec"

BTIMDRFT values are different...FAIL...
BTIMNULL values are different...FAIL...
BTIMRATE values are different...FAIL...
omit - DEC_PNT values different more than 0.000300
omit - FP_TEMP values different more than 2.000000
omit - RA_PNT values different more than 0.000300
omit - ROLL_NOM values different more than 1.000000
omit - ROLL_PNT values different more than 1.000000
omit - SIM_Z values different more than 0.100000
BTIMDRFT values are different...FAIL...
BTIMNULL values are different...FAIL...
BTIMRATE values are different...FAIL...
omit - DEC_PNT values different more than 0.000300
omit - FP_TEMP values different more than 2.000000
omit - RA_PNT values different more than 0.000300
omit - ROLL_NOM values different more than 1.000000
omit - ROLL_PNT values different more than 1.000000
omit - SIM_Z values different more than 0.100000


(c) Finally, we detected point sources in 0.5-2 and 2-7 keV.

In [26]:
punlearn wavdetect

wavdetect infile=merged4/0.5-2.0_thresh.img psffile=merged4/expweighted_0.5-2.0.psf \
    expfile=merged4/0.5-2.0_thresh.expmap outfile=merged4/src_0.5-2.0.fits \
    scellfile=merged4/scell_0.5-2.0.fits imagefile=merged4/src_0.5-2.0.img \
    defnbkgfile=merged4/nbkg_0.5-2.0.fits regfile=merged4/src_0.5-2.0.reg \
    scales="1.0 2.0 4.0 8.0 16.0 32.0" sigthresh=1e-4 ellsigma=5.0 clobber+

wavdetect infile=merged4/2.0-7.0_thresh.img psffile=merged4/expweighted_2.0-7.0.psf \
    expfile=merged4/2.0-7.0_thresh.expmap outfile=merged4/src_2.0-7.0.fits \
    scellfile=merged4/scell_2.0-7.0.fits imagefile=merged4/src_2.0-7.0.img \
    defnbkgfile=merged4/nbkg_2.0-7.0.fits regfile=merged4/src_2.0-7.0.reg \
    scales="1.0 2.0 4.0 8.0 16.0 32.0" sigthresh=1e-4 ellsigma=5.0 clobber+

(d) The point sources detected by <i>wavdetect</i> were checked visually, and point sources in the brightest parts of the ICM were checked against the VLA and HST images. Fake detections were removed, while missed point sources were added to the final region definition. The final point source region file was saved as `src_J2000.reg` in J2000 coordinates, and `src_phys.reg` in physical coordinates (the latter as an exclusion file, i.e. field() - circle(params)).

### 6. Remove Point Sources

Point sources were removed from the source and stowed background event files of each ObsID. Additionally, the edges of the ACIS-I detector were removed in regions of width ~30 arcsec. Only the ACIS-I CCDs were used in the analysis, so any other CCDs that were on during the observations were removed from the final event files.

In [33]:
punlearn dmcopy

for obsid in $obsid1 $obsid2 $obsid3 $obsid4
do
    dmcopy "merged4/${obsid}_reproj_evt.fits[ccd_id=0:3][sky=region(merged4/${obsid}_src_edges_phys.reg)]" \
        merged4/${obsid}_goodreg.evt opt=all clobber+
    dmcopy "merged4/${obsid}_bgstow_clean.evt[ccd_id=0:3][sky=region(merged4/${obsid}_src_edges_phys.reg)]" \
        merged4/${obsid}_bgstow_goodreg.evt opt=all clobber+
done

### 7. Images Free of Point Sources

The imaging analysis requires images free of point sources. These images are created below in the energy band 0.5-4 keV. 

First, we created a spectral weights file that will be used by <i>flux_obs</i> to create weighted instrument maps. The maps are used (behind the scenes, also by <i>flux_obs</i>) when generating the exposure maps for each of the ObsIDs.

In [3]:
punlearn make_instmap_weights
make_instmap_weights merged4/weights_500-4000.txt "xsphabs.gal*xsapec.th" \
    paramvals="gal.nh=0.084;th.kT=12.0;th.abundanc=0.20;th.redshift=0.5458" \
    emin=0.5 emax=4.0 ewidth=0.25 clobber+

Model used: 
(xsphabs.gal * xsapec.th)
   Param        Type          Value          Min          Max      Units
   -----        ----          -----          ---          ---      -----
   gal.nH       thawed        0.084            0       100000 10^22 atoms / cm^2
   th.kT        thawed           12        0.008           64        keV
   th.Abundanc  frozen          0.2            0            5           
   th.redshift  frozen       0.5458       -0.999           10           
   th.norm      thawed            1            0        1e+24           
Created: merged4/weights_500-4000.txt


Merged point source-free images with spatial resolutions of 0.5, 1, and 2 arcsec were then created using <i>flux_obs</i>. Running the cell below will likely take several minutes.

In [4]:
punlearn flux_obs
for bin in "1" "2" "4"
do
    flux_obs "merged4/${obsid1}_goodreg.evt,merged4/${obsid2}_goodreg.evt,\
              merged4/${obsid3}_goodreg.evt,merged4/${obsid4}_goodreg.evt" \
        merged4/srcfree_bin${bin}_500-4000_ bands=merged4/weights_500-4000.txt \
        binsize=${bin} maskfiles="merged4/${obsid1}.mask,merged4/${obsid2}.mask,\
                                  merged4/${obsid3}.mask,merged4/${obsid4}.mask" \
        badpixfiles="merged4/${obsid1}.bpix,merged4/${obsid2}.bpix, \
                     merged4/${obsid3}.bpix,merged4/${obsid4}.bpix" \
        expmapthresh="3%" clobber+
done

Running flux_obs
Version: 05 January 2016

Verifying 4 observations.
band1 uses weights file merged4/weights_500-4000.txt (0.5 to 4.0 keV)
Calculating the output grid

The merged images will have 3734 by 3412 pixels, a pixel size of 0.492 arcsec,
    and cover x=2379.5:6113.5:1, y=2469.5:5881.5:1.

Creating the fluxed images for 4 observations in parallel.
# DMCOPY (CIAO 4.8): WARNING: Creating large image: 102 MB. Current max set at 50 MB.
Increase maximum using [opt mem=n] or increase blocking to reduce size.
Creating aspect histograms for obsid 1655
Creating aspect histograms for obsid 16235
Creating aspect histograms for obsid 4200
# DMCOPY (CIAO 4.8): WARNING: Creating large image: 102 MB. Current max set at 50 MB.
Increase maximum using [opt mem=n] or increase blocking to reduce size.
# DMCOPY (CIAO 4.8): WARNING: Creating large image: 102 MB. Current max set at 50 MB.
Increase maximum using [opt mem=n] or increase blocking to reduce size.
Creating aspect histograms for obsid 163

Regions contaminated by point sources are not automatically excluded from the exposure maps. However, when modeling surface brightness profiles, it is important to exclude these regions in order to correctly calculate the surface brightness. If a pixel is 0 in the source image, it could be 0 because no photon hit it. However, it could also be 0 because it is part of a region contaminated by a point source and excluded from the image (dmcopy sets the pixels in excluded regions to 0). On the other hand, the exposure images are non-zero at all the pixels of the CCDs. Therefore, to differentiate between these two cases in which a pixel is 0, the regions contaminated by point sources are excluded from the exposure maps.

In [35]:
punlearn dmcopy

for bin in "bin1" "bin2" "bin4"
do
    for obsid in ${obsid1} ${obsid2} ${obsid3} ${obsid4}
    do
        expmap="merged4/srcfree_${bin}_500-4000_${obsid}_band1_thresh.expmap"
        regfile="merged4/${obsid}_src_edges_phys.reg"
        dmcopy "$expmap[sky=region($regfile)]" \
            merged4/srcfree_${bin}_500-4000_${obsid}_thresh.expmap_nosrcedg clobber+
    done
done


In [36]:
for bin in "bin1" "bin2" "bin4"
do
    dmimgcalc "merged4/srcfree_${bin}_500-4000_${obsid1}_thresh.expmap_nosrcedg, \
               merged4/srcfree_${bin}_500-4000_${obsid2}_thresh.expmap_nosrcedg, \
               merged4/srcfree_${bin}_500-4000_${obsid3}_thresh.expmap_nosrcedg, \
               merged4/srcfree_${bin}_500-4000_${obsid4}_thresh.expmap_nosrcedg" \
        none merged4/srcfree_${bin}_500-4000_thresh.expmap_nosrcedg \
        op="imgout=img1+img2+img3+img4" clobber+
done

BTIMDRFT values are different...FAIL...
BTIMNULL values are different...FAIL...
BTIMRATE values are different...FAIL...
omit - DEC_PNT values different more than 0.000300
omit - FP_TEMP values different more than 2.000000
omit - RA_PNT values different more than 0.000300
omit - ROLL_NOM values different more than 1.000000
omit - ROLL_PNT values different more than 1.000000
omit - SIM_Z values different more than 0.100000
BTIMDRFT values are different...FAIL...
BTIMNULL values are different...FAIL...
BTIMRATE values are different...FAIL...
omit - DEC_PNT values different more than 0.000300
omit - FP_TEMP values different more than 2.000000
omit - RA_PNT values different more than 0.000300
omit - ROLL_NOM values different more than 1.000000
omit - ROLL_PNT values different more than 1.000000
omit - SIM_Z values different more than 0.100000
BTIMDRFT values are different...FAIL...
BTIMNULL values are different...FAIL...
BTIMRATE values are different...FAIL...
omit - DEC_PNT values differen

We also created stowed background images free of point sources. These images have a different exposure time than the observations, so in the next step they will be renormalized to have the same 10-12 keV count rate as the corresponding ObsIDs.

In [37]:
punlearn dmcopy

bin1="x=2379.5:6113.5:1,y=2469.5:5881.5:1"
bin2="x=2378.5:6114.5:2,y=2468.5:5882.5:2"
bin4="x=2376.5:6116.5:4,y=2468.5:5884.5:4"

for obsid in ${obsid1} ${obsid2} ${obsid3} ${obsid4}
do
    dmcopy "merged4/${obsid}_bgstow_goodreg.evt[energy=500:4000][bin ${bin1}]" \
        merged4/${obsid}_bin1_500-4000_bgstow_goodreg.img opt=image clobber+
    dmcopy "merged4/${obsid}_bgstow_goodreg.evt[energy=500:4000][bin ${bin2}]" \
        merged4/${obsid}_bin2_500-4000_bgstow_goodreg.img opt=image clobber+
    dmcopy "merged4/${obsid}_bgstow_goodreg.evt[energy=500:4000][bin ${bin4}]" \
        merged4/${obsid}_bin4_500-4000_bgstow_goodreg.img opt=image clobber+
done

### 8. Calculate Background Normalizations

The cell below prints the counts in a region that excludes point sources, cluster emission, and the edges of the ACIS-I detector. The exclusion regions were saved in the files `ObsID_skybkg.reg`, which can be found in the `merged4` directory. The cell also prints the exposure time of the ObsID and the stowed background event files.

In [21]:
for obsid in ${obsid1} ${obsid2} ${obsid3} ${obsid4}
do
    printf "ObsID ${obsid} 10-12 keV counts and exp time:\n"
    dmstat "merged4/${obsid}_goodreg.evt[sky=region(merged4/${obsid}_skybkg.reg)][energy=10000:12000][col grade]" \
        centroid=no | grep -i "good:"
    expo=`dmkeypar "merged4/${obsid}_goodreg.evt[EVENTS]" EXPOSURE echo+`
    printf "Exposure time source (s): $expo\n\n"
    printf "Bgstow for ObsID ${obsid} -- 10-12 keV counts and exp time:\n"
    dmstat "merged4/${obsid}_bgstow_goodreg.evt[sky=region(merged4/${obsid}_skybkg.reg)][energy=10000:12000][col grade]" \
        centroid=no | grep -i "good:"
    expo=`dmkeypar "merged4/${obsid}_bgstow_goodreg.evt[EVENTS]" EXPOSURE echo+`
    printf "Exposure time background (s): $expo\n\n\n\n"
done

ObsID 16235 10-12 keV counts and exp time:
   good:	5073 
Exposure time source (s): 61579.805663854

Bgstow for ObsID 16235 -- 10-12 keV counts and exp time:
   good:	27642 
Exposure time background (s): 240352.59999996



ObsID 16305 10-12 keV counts and exp time:
   good:	3940 
Exposure time source (s): 79756.659791342

Bgstow for ObsID 16305 -- 10-12 keV counts and exp time:
   good:	17136 
Exposure time background (s): 240352.59999996



ObsID 4200 10-12 keV counts and exp time:
   good:	1867 
Exposure time source (s): 51633.841987317

Bgstow for ObsID 4200 -- 10-12 keV counts and exp time:
   good:	18762 
Exposure time background (s): 415000.0



ObsID 1655 10-12 keV counts and exp time:
   good:	1622 
Exposure time source (s): 15770.495280984

Bgstow for ObsID 1655 -- 10-12 keV counts and exp time:
   good:	54373 
Exposure time background (s): 415000.0





The correction factors were calculated as:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; `norm_factor = counts_ObsID / counts_bgstow * exp_bgstow / exp_ObsID `

This yields the following normalization factors for the four ObsIDs:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **ObsID 16235:** 0.71632<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **ObsID 16305:** 0.69290<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **ObsID 4200:** 0.79980<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **ObsID 1655:** 0.78500<br>

In [2]:
rf1=0.71632
rf2=0.69290
rf3=0.79980
rf4=0.78500

The normalization factors are also added to the headers of the background images, using the keyword BKGNORM. This keyword will later be used by SurfFit to normalize the background profile and correctly calculate background uncertainties.

In [5]:
dmhedit "merged4/${obsid1}_bin4_500-4000_bgstow_goodreg.img" none \
    add BKGNORM "$rf1 / [s] Background normalization" datatype=float
    
dmhedit "merged4/${obsid2}_bin4_500-4000_bgstow_goodreg.img" none \
    add BKGNORM "$rf2 / [s] Background normalization" datatype=float
    
dmhedit "merged4/${obsid3}_bin4_500-4000_bgstow_goodreg.img" none \
    add BKGNORM "$rf3 / [s] Background normalization" datatype=float
    
dmhedit "merged4/${obsid4}_bin4_500-4000_bgstow_goodreg.img" none \
    add BKGNORM "$rf4 / [s] Background normalization" datatype=float

### 9. Create Merged Background Image

The individual stowed background images in 0.5-4 keV were normalized by the factors calculated above, and then added into a single image.

In [38]:
for i in 1 2 3 4
do
    obsid_name="obsid"$i
    obsid=${!obsid_name}
    rf_name="rf"$i
    rf=${!rf_name}
    exptime_source=`dmkeypar merged4/srcfree_bin1_500-4000_${obsid}_band1_thresh.img EXPOSURE echo+`
    exptime_bgstow=`dmkeypar merged4/${obsid}_bin1_500-4000_bgstow_goodreg.img EXPOSURE echo+`
    corr=`awk "BEGIN {print $exptime_source/$exptime_bgstow*$rf}"`
    dmimgcalc merged4/${obsid}_bin1_500-4000_bgstow_goodreg.img none \
        merged4/srcfree_bin1_500-4000_${obsid}_bgstow_renorm.img add weight=$corr clobber+
    dmimgcalc merged4/${obsid}_bin2_500-4000_bgstow_goodreg.img none \
        merged4/srcfree_bin2_500-4000_${obsid}_bgstow_renorm.img add weight=$corr clobber+
    dmimgcalc merged4/${obsid}_bin4_500-4000_bgstow_goodreg.img none \
        merged4/srcfree_bin4_500-4000_${obsid}_bgstow_renorm.img add weight=$corr clobber+
done

In [39]:
for bin in "bin1" "bin2" "bin4"
do
    dmimgcalc "merged4/srcfree_${bin}_500-4000_${obsid1}_bgstow_renorm.img, \
               merged4/srcfree_${bin}_500-4000_${obsid2}_bgstow_renorm.img, \
               merged4/srcfree_${bin}_500-4000_${obsid3}_bgstow_renorm.img, \
               merged4/srcfree_${bin}_500-4000_${obsid4}_bgstow_renorm.img" \
        none merged4/srcfree_${bin}_500-4000_bgstow_renorm.img \
        op="imgout=img1+img2+img3+img4" clobber+
done

omit - DEC_PNT values different more than 0.000300
omit - RAND_SKY values different more than 0.050000
omit - RA_PNT values different more than 0.000300
omit - ROLL_PNT values different more than 1.000000
omit - SIM_Z values different more than 0.100000
omit - DEC_PNT values different more than 0.000300
omit - RAND_SKY values different more than 0.050000
omit - RA_PNT values different more than 0.000300
omit - ROLL_PNT values different more than 1.000000
omit - SIM_Z values different more than 0.100000
omit - DEC_PNT values different more than 0.000300
omit - RAND_SKY values different more than 0.050000
omit - RA_PNT values different more than 0.000300
omit - ROLL_PNT values different more than 1.000000
omit - SIM_Z values different more than 0.100000


We'll also need an image of the background uncertainties, since simply taking the square root of the background image is not going to yield correct uncertainties. The image created below has the squares of the background uncertainties.

In [4]:
for i in 1 2 3 4
do
    obsid_name="obsid"$i
    obsid=${!obsid_name}
    rf_name="rf"$i
    rf=${!rf_name}
    exptime_source=`dmkeypar merged4/srcfree_bin1_500-4000_${obsid}_band1_thresh.img EXPOSURE echo+`
    exptime_bgstow=`dmkeypar merged4/${obsid}_bin1_500-4000_bgstow_goodreg.img EXPOSURE echo+`
    corr=`awk "BEGIN {print $exptime_source/$exptime_bgstow*$exptime_source/$exptime_bgstow*$rf*$rf}"`
    dmimgcalc merged4/${obsid}_bin1_500-4000_bgstow_goodreg.img none \
        merged4/srcfree_bin1_500-4000_${obsid}_bgstow_err_renorm.img add weight=$corr clobber+
    dmimgcalc merged4/${obsid}_bin2_500-4000_bgstow_goodreg.img none \
        merged4/srcfree_bin2_500-4000_${obsid}_bgstow_err_renorm.img add weight=$corr clobber+
    dmimgcalc merged4/${obsid}_bin4_500-4000_bgstow_goodreg.img none \
        merged4/srcfree_bin4_500-4000_${obsid}_bgstow_err_renorm.img add weight=$corr clobber+
done

for bin in "bin1" "bin2" "bin4"
do
    dmimgcalc "merged4/srcfree_${bin}_500-4000_${obsid1}_bgstow_err_renorm.img, \
               merged4/srcfree_${bin}_500-4000_${obsid2}_bgstow_err_renorm.img, \
               merged4/srcfree_${bin}_500-4000_${obsid3}_bgstow_err_renorm.img, \
               merged4/srcfree_${bin}_500-4000_${obsid4}_bgstow_err_renorm.img" \
        none merged4/srcfree_${bin}_500-4000_bgstow_err_renorm.img \
        op="imgout=img1+img2+img3+img4" clobber+
done

omit - DEC_PNT values different more than 0.000300
omit - RAND_SKY values different more than 0.050000
omit - RA_PNT values different more than 0.000300
omit - ROLL_PNT values different more than 1.000000
omit - SIM_Z values different more than 0.100000
omit - DEC_PNT values different more than 0.000300
omit - RAND_SKY values different more than 0.050000
omit - RA_PNT values different more than 0.000300
omit - ROLL_PNT values different more than 1.000000
omit - SIM_Z values different more than 0.100000
omit - DEC_PNT values different more than 0.000300
omit - RAND_SKY values different more than 0.050000
omit - RA_PNT values different more than 0.000300
omit - ROLL_PNT values different more than 1.000000
omit - SIM_Z values different more than 0.100000


### 10. Background Modeling

Sky background+foreground spectra were extracted from regions free of point sources and ICM emission. The stowed background spectra associated with each of the sky spectra were normalized by the factors calculated above; the normalization was applied to the BACKSCAL keywords. The area (in units of arcmin\*\*2) from which the spectra were extracted was written in the keyword AREASCAL for both the sky and the stowed background spectra, which means the `XSpec` models will have normalizations in units of &lt;default XSpec units&gt;/arcmin\*\*2.

In [5]:
punlearn specextract
for i in "1" "2" "3" "4"
do
    obsid_name="obsid"$i
    obsid=${!obsid_name}
    rf_name="rf"$i
    rf=${!rf_name}
#    specextract infile="merged4/${obsid}_goodreg.evt[sky=region(merged4/${obsid}_skybkg.reg)]" \
#        outroot=merged4/${obsid}_skybkg \
#        bkgfile="merged4/${obsid}_bgstow_goodreg.evt[sky=region(merged4/${obsid}_skybkg.reg)]" \
#        asp=@${obsid}.aspect mskfile=merged4/${obsid}.mask badpixfile=merged4/${obsid}.bpix \
#        weight=yes correct=no grouptype=NUM_CTS binspec=1 bkgresp=no clobber=yes
    dmkeypar "merged4/${obsid}_skybkg.pi[SPECTRUM]" BACKSCAL echo+ >& temp
    NewBACKSCAL=`awk '{printf "%e\n",$0/a}' a=${rf} OFS=, temp`
    dmhedit "merged4/${obsid}_skybkg_bkg.pi[SPECTRUM]" none add key=BACKSCAL value=$NewBACKSCAL
    NewAREASCAL=`awk '{printf "%e\n",$0*a}' a=4512.40001536 OFS=, temp`
    dmhedit "merged4/${obsid}_skybkg.pi[SPECTRUM]" none add key=AREASCAL value=$NewAREASCAL
    dmhedit "merged4/${obsid}_skybkg_bkg.pi[SPECTRUM]" none add key=AREASCAL value=$NewAREASCAL
    dmhedit "merged4/${obsid}_skybkg_grp.pi[SPECTRUM]" none add key=AREASCAL value=$NewAREASCAL
    rm -f temp
done

The spectra were fitted in `XSpec` with a model consisting of unabsorbed thermal emission from the Local Hot Bubble, absorbed thermal emission from the Galactic Halo, and absorbed power-law emission from unresolved point sources:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; `APEC + (APEC + power-law) * phabs`.

In this composite model, all the redshifts were fixed to 0, the metallicties to solar (assuming the abundace table of Ferland 1992), and [the absorption to 8.36e20 cm\*\*-2](http://www.swift.ac.uk/analysis/nhtot/index.php). The index of the power-law was fixed to 1.41 (De Luca & Molendi 2004). The power-law normalizations were linked for ObsIDs 16235 and 16305, because these observations point almost in the same region of the sky; the normalizations for the other two ObsIDs were untied. The normalizations of the thermal components were tied for all ObsIDs.

The fit was done using the C-statistic (Wachter 1979).

The best-fitting parameters obtained were (with 1-sigma uncertainties):

`GH temperature                               0.50939      0.679356       (-0.0830017,0.0869636)
GH normalization                             2.34089e-07  3.23616e-07    (-4.40694e-08,4.54579e-08)
PL normalization for ObsIDs 16235 and 16305  4.07855e-07  4.79302e-07    (-3.65226e-08,3.49236e-08)
PL normalization for ObsID 4200              6.43973e-07  7.50144e-07    (-5.76677e-08,4.85034e-08)`

The fitting parameters were defined in the file `merged4/bkg.xcm`, which is included in the repository. The log file associated with this Xspec file is `merged4/bkg.log`, which is also available. We noticed possible residual soft proton contamination in ObsID 1655, so this ObsID was excluded from the analysis. The best-fitting parameters listed above were obtained from the remaining three ObsIDs.

### 11. Large-Scale SE Filament

We examined the surface brightness along the filament in a box region defined as:

`box(7:17:48.681,+37:42:57.79,58.8752",393.408",134)`.

This region includes part of the cluster ICM, and the group located along the filament. Therefore, we used the surface brightness profile to identify the subregion where the profile was flat, which would be indicative of it containing only filament emission. The surface brightness does not perfectly flatten between the cluster and the filament. However, it is almost flat in the region:

`box(7:17:47.661,+37:43:09.31,58.572",87.8177",134)`.

Hence, we used this region to extract a spectrum of the filament. Nevertheless, this region might be contaminated by cluster emission. To verify this and possibly remove the ICM contribution, we also extracted spectra in a box region located W of the filament region:

`box(7:17:38.303,+37:41:29.44,222.972",106.291",134)`.

In [11]:
punlearn specextract
for i in "1" "2" "3" "4"
do
    for reg in "filament-w" "filament"
    do
        obsid_name="obsid"$i
        obsid=${!obsid_name}
        rf_name="rf"$i
        rf=${!rf_name}
        specextract infile="merged4/${obsid}_goodreg.evt[sky=region(merged4/$reg.reg)]" \
            outroot=merged4/${obsid}_$reg \
            bkgfile="merged4/${obsid}_bgstow_goodreg.evt[sky=region(merged4/$reg.reg)]" \
            asp=@${obsid}.aspect mskfile=merged4/${obsid}.mask badpixfile=merged4/${obsid}.bpix \
            weight=yes correct=no grouptype=NUM_CTS binspec=1 bkgresp=no clobber=yes
        dmkeypar "merged4/${obsid}_${reg}.pi[SPECTRUM]" BACKSCAL echo+ >& temp
        NewBACKSCAL=`awk '{printf "%e\n",$0/a}' a=${rf} OFS=, temp`
        dmhedit "merged4/${obsid}_${reg}_bkg.pi[SPECTRUM]" none add key=BACKSCAL value=$NewBACKSCAL
        NewAREASCAL=`awk '{printf "%e\n",$0*a}' a=4512.40001536 OFS=, temp`
        dmhedit "merged4/${obsid}_${reg}.pi[SPECTRUM]" none add key=AREASCAL value=$NewAREASCAL
        dmhedit "merged4/${obsid}_${reg}_bkg.pi[SPECTRUM]" none add key=AREASCAL value=$NewAREASCAL
        dmhedit "merged4/${obsid}_${reg}_grp.pi[SPECTRUM]" none add key=AREASCAL value=$NewAREASCAL
        rm -f temp
    done
done

  Running: specextract
  Version:    09 November 2015
Checking for blank sky background files...


Setting bad pixel file 

Extracting src spectra 

Creating src ARF 

Creating src RMF 

Using mkacisrmf...

Grouping src spectrum 

Updating header of merged4/16235_filament-w.pi with RESPFILE and ANCRFILE keywords.

Updating header of merged4/16235_filament-w_grp.pi with RESPFILE and ANCRFILE keywords.

Setting bad pixel file 

Extracting bkg spectra 


Updating header of merged4/16235_filament-w.pi with BACKFILE keyword.

Updating header of merged4/16235_filament-w_grp.pi with BACKFILE keyword.

  Running: specextract
  Version:    09 November 2015
Checking for blank sky background files...


Setting bad pixel file 

Extracting src spectra 

Creating src ARF 

Creating src RMF 

Using mkacisrmf...

Grouping src spectrum 

Updating header of merged4/16305_filament-w.pi with RESPFILE and ANCRFILE keywords.

Updating header of merged4/16305_filament-w_grp.pi with RESPFILE and ANCRFILE keyw

The spectra extracted above were fitted with a model consisting of sky background emission (whose parameters were fixed to the best-fitting values determined above) and thermal emission from the ICM/filament, i.e.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; `APEC + (APEC + power-law + APEC) * phabs`.

For the ICM/filament component, the redshift was fixed to 0.546 and the metallicity was originally fixed to 0.2 solar. We assumed the ICM properties to be the same W and E of the filament, so the temperatures and normalizations for these spectra were linked in the fit. The best-fitting parameters were found to be (again with 1-sigma uncertainties):

##### Filament:

`Temperature          2.99482      5.18358        (-0.959026,1.22973)
Normalization        4.11539e-05  5.10886e-05    (-4.46512e-06,5.46957e-06)`

##### W of the Filament

`Temperature          5.01189      13.5261        (-2.46303,6.05122)
Normalization        1.43889e-05  1.70644e-05    (-1.24561e-06,1.42993e-06)`


Because the ICM flux is clearly not 0 at the location of the filament, we performed another fit in which we modeled the filament with the sum of thermal emission from the filament and thermal emission from the ICM. In this fit, we simultaneously modeled the spectra W of the filament and the filament spectra. The best-fitting parameters are listed below:

##### Filament:

`Temperature          1.32742      2.09035        (-0.250027,0.512902)
Normalization        3.39348e-05  4.55738e-05    (-6.0474e-06,5.59153e-06)`

##### W of the Filament

`Temperature          7.72584      20.7628        (-3.94875,9.08823)
Normalization        1.45026e-05  1.68793e-05    (-1.0259e-06,1.35081e-06)`

Abundances of large-scale cosmic filaments are very uncertain. They could be ~0.2 solar, similar to the abundances at the virial radii of galaxy clusters, or could decrease close to 0. Therefore, we also performed the fit above assuming filament metallicities of 0 and 0.1 solar. The best-fitting parameters were:

#### Metallicity Z = 0.1 solar

##### Filament:

`Temperature          1.24258      2.10665        (-0.27294,0.591133)
Normalization        3.7239e-05   5.62527e-05    (-9.97053e-06,9.04315e-06)`

##### W of the Filament

`Temperature          7.69544      20.5753        (-4.27099,8.60885)
Normalization        1.45329e-05  1.68823e-05    (-1.05989e-06,1.28954e-06)`


#### Metallicity Z = 0

##### Filament:

`Temperature          1.24889      3.40061        (-0.687375,1.46435)
Normalization        3.07955e-05  7.02732e-05    (-1.4326e-05,2.51517e-05)`

##### W of the Filament

`Temperature          6.0074       17.5577        (-5.22587,6.32448)
Normalization        1.43169e-05  1.66366e-05    (-1.14315e-06,1.1766e-06)`

**We conclude that the effect of the unknown metallicity is small compared to the much larger statistical uncertainties.**

### 12. Density of the Filament

The `XSpec` normalizations listed above are defined as:

`N = 1e-14/pi/S/D**2/(1+z)**2 * n_e * n_H * V`

where:

S - projected area of the region from which the spectrum was extracted
D - angular size distance at the cluster redshift
z - cluster redshift
n_e - electron number density
n_H - hydrogen number density
V - volume of the region from which the spectrum was extracted

and we assumed the density to be constant within the regions of interest.

To calculate the density of the gas within the region of the filament from which our spectra were extracted, we assumed an elliptic cylinder geometry. Jauzac et al. (2012) found that the filament is inclined at 75 degrees with respect to the plane of the sky, and has a diameter of about 1.6 Mpc and a length of about 19 Mpc (as in the paper, we assumed H0 = 70 km/s/Mpc, omega_m = 0.3, and omega_v = 0.7). Therefore, we assumed our region is an elliptical cylinder with a length L = 2.2 Mpc (~87.8" / cos(75 deg)), and a base with a major axis a = 0.8 Mpc and a minor axis b = 0.2 Mpc (~29.3"). The volume of this region is:

`V = pi * L * a * b = 1.1 Mpc**3`

We assumed `n_H = 1.2 * n_e` (e.g. Böhringer and Werner 2010). In our cosmology, the angular size distance at the cluster redshift is 1316.4 Mpc. Therefore, for an `XSpec` normalization of ~3e-5 cm\*\*-5 arcmin\*\*-2 and a projected region area of 1.4 arcmin\*\*2, we found an electron number density of ~1e-4 cm\*\*-3. This is equivalent with a baryon mass of ~3e12 solar masses. Assuming a baryon fraction of 0.044 (Kirkman et al. 2003), the filament is over-dense by a factor of ~220 compared to the mean baryon density of the Universe. Furthermore, assuming a baryon fraction of 0.15 (Mantz et al. 2014), the total mass of the filament in the region from which our spectra were extracted is ~2e13 solar masses. Accounting for the full length of the filament and assuming a cylindrical geometry, the total mass of the filament is ~7e14 solar masses.

### 13. Group in the Filament

In [8]:
reg="group"
punlearn specextract
for i in "1" "2" "3" "4"
do
    obsid_name="obsid"$i
    obsid=${!obsid_name}
    rf_name="rf"$i
    rf=${!rf_name}
    specextract infile="merged4/${obsid}_goodreg.evt[sky=region(merged4/$reg.reg)]" \
        outroot=merged4/${obsid}_$reg \
        bkgfile="merged4/${obsid}_bgstow_goodreg.evt[sky=region(merged4/$reg.reg)]" \
        asp=@${obsid}.aspect mskfile=merged4/${obsid}.mask badpixfile=merged4/${obsid}.bpix \
        weight=yes correct=no grouptype=NUM_CTS binspec=1 bkgresp=no clobber=yes
    dmkeypar "merged4/${obsid}_${reg}.pi[SPECTRUM]" BACKSCAL echo+ >& temp
    NewBACKSCAL=`awk '{printf "%e\n",$0/a}' a=${rf} OFS=, temp`
    dmhedit "merged4/${obsid}_${reg}_bkg.pi[SPECTRUM]" none add key=BACKSCAL value=$NewBACKSCAL
    NewAREASCAL=`awk '{printf "%e\n",$0*a}' a=4512.40001536 OFS=, temp`
    dmhedit "merged4/${obsid}_${reg}.pi[SPECTRUM]" none add key=AREASCAL value=$NewAREASCAL
    dmhedit "merged4/${obsid}_${reg}_bkg.pi[SPECTRUM]" none add key=AREASCAL value=$NewAREASCAL
    dmhedit "merged4/${obsid}_${reg}_grp.pi[SPECTRUM]" none add key=AREASCAL value=$NewAREASCAL
    rm -f temp
done

  Running: specextract
  Version:    09 November 2015
Checking for blank sky background files...


Setting bad pixel file 

Extracting src spectra 

Creating src ARF 

Creating src RMF 

Using mkacisrmf...

Grouping src spectrum 

Updating header of merged4/16235_group.pi with RESPFILE and ANCRFILE keywords.

Updating header of merged4/16235_group_grp.pi with RESPFILE and ANCRFILE keywords.

Setting bad pixel file 

Extracting bkg spectra 


Updating header of merged4/16235_group.pi with BACKFILE keyword.

Updating header of merged4/16235_group_grp.pi with BACKFILE keyword.

  Running: specextract
  Version:    09 November 2015
Checking for blank sky background files...


Setting bad pixel file 

Extracting src spectra 

Creating src ARF 

Creating src RMF 

Using mkacisrmf...

Grouping src spectrum 

Updating header of merged4/16305_group.pi with RESPFILE and ANCRFILE keywords.

Updating header of merged4/16305_group_grp.pi with RESPFILE and ANCRFILE keywords.

Setting bad pixel file 

In [3]:
punlearn specextract
for i in "1" "2" "3" "4"
do
    for reg in "core-n-alt" #"core" "core-front" "core-behind" "core-n" "core-s"
    do
        obsid_name="obsid"$i
        obsid=${!obsid_name}
        rf_name="rf"$i
        rf=${!rf_name}
        specextract infile="merged4/${obsid}_goodreg.evt[sky=region(merged4/$reg.reg)]" \
            outroot=merged4/${obsid}_$reg \
            bkgfile="merged4/${obsid}_bgstow_goodreg.evt[sky=region(merged4/$reg.reg)]" \
            asp=@${obsid}.aspect mskfile=merged4/${obsid}.mask badpixfile=merged4/${obsid}.bpix \
            weight=yes correct=no grouptype=NUM_CTS binspec=1 bkgresp=no clobber=yes
        dmkeypar "merged4/${obsid}_${reg}.pi[SPECTRUM]" BACKSCAL echo+ >& temp
        NewBACKSCAL=`awk '{printf "%e\n",$0/a}' a=${rf} OFS=, temp`
        dmhedit "merged4/${obsid}_${reg}_bkg.pi[SPECTRUM]" none add key=BACKSCAL value=$NewBACKSCAL
        NewAREASCAL=`awk '{printf "%e\n",$0*a}' a=4512.40001536 OFS=, temp`
        dmhedit "merged4/${obsid}_${reg}.pi[SPECTRUM]" none add key=AREASCAL value=$NewAREASCAL
        dmhedit "merged4/${obsid}_${reg}_bkg.pi[SPECTRUM]" none add key=AREASCAL value=$NewAREASCAL
        dmhedit "merged4/${obsid}_${reg}_grp.pi[SPECTRUM]" none add key=AREASCAL value=$NewAREASCAL
        rm -f temp
    done
done

  Running: specextract
  Version:    09 November 2015
Checking for blank sky background files...


chipy=1025 for ACIS-3; using chipy=1024 for FEF and RMF look up.

Setting bad pixel file 

Extracting src spectra 

Creating src ARF 

Creating src RMF 

Using mkacisrmf...

Grouping src spectrum 

Updating header of merged4/16235_core-n-alt.pi with RESPFILE and ANCRFILE keywords.

Updating header of merged4/16235_core-n-alt_grp.pi with RESPFILE and ANCRFILE keywords.

Setting bad pixel file 

Extracting bkg spectra 


Updating header of merged4/16235_core-n-alt.pi with BACKFILE keyword.

Updating header of merged4/16235_core-n-alt_grp.pi with BACKFILE keyword.

  Running: specextract
  Version:    09 November 2015
Checking for blank sky background files...


Setting bad pixel file 

Extracting src spectra 

Creating src ARF 

Creating src RMF 

Using mkacisrmf...

Grouping src spectrum 

Updating header of merged4/16305_core-n-alt.pi with RESPFILE and ANCRFILE keywords.

Updating header 